In [ ]:
from pathlib import Path
Path("src").mkdir(exist_ok=True)

In [ ]:
import digitalhub as dh
project = dh.get_or_create_project("project-ml-ci")

In [ ]:
%%writefile "src/data-prep.py"

import pandas as pd
from sklearn.datasets import load_breast_cancer

from digitalhub_runtime_python import handler

@handler(outputs=["dataset"])
def breast_cancer_generator():
    """
    A function which generates the breast cancer dataset
    """
    breast_cancer = load_breast_cancer()
    breast_cancer_dataset = pd.DataFrame(
        data=breast_cancer.data, columns=breast_cancer.feature_names
    )
    breast_cancer_labels = pd.DataFrame(data=breast_cancer.target, columns=["target"])
    breast_cancer_dataset = pd.concat(
        [breast_cancer_dataset, breast_cancer_labels], axis=1
    )

    return breast_cancer_dataset

In [ ]:
data_gen_fn = project.new_function(name="data-prep",
                                   kind="python",
                                   python_version="PYTHON3_10",
                                   code_src="src/data-prep.py",
                                   handler="breast_cancer_generator")

In [ ]:
gen_data_run = data_gen_fn.run("job", wait=True)

Wait for data manipulation job to be completed. One can view the state of run from console or using the digitalhub sdk API

In [ ]:
gen_data_run.refresh().status.state

Once 'Completed', proceed with next steps.

In [ ]:
gen_data_run.output("dataset").as_df().head()

## Training

In [ ]:
%%writefile "src/train-model.py"


import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from digitalhub_runtime_python import handler
from sklearn.svm import SVC
from pickle import dump
import sklearn.metrics
import os

@handler(outputs=["model"])
def train(project, di):

    df_cancer = di.as_df()
    X = df_cancer.drop(['target'],axis=1)
    y = df_cancer['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=5)
    svc_model = SVC()
    svc_model.fit(X_train, y_train)
    y_predict = svc_model.predict(X_test)

    if not os.path.exists("model"):
        os.makedirs("model")

    with open("model/cancer_classifier.pkl", "wb") as f:
        dump(svc_model, f, protocol=5)

    metrics = {
        "f1_score": sklearn.metrics.f1_score(y_test, y_predict),
        "accuracy": sklearn.metrics.accuracy_score(y_test, y_predict),
        "precision": sklearn.metrics.precision_score(y_test, y_predict),
        "recall": sklearn.metrics.recall_score(y_test, y_predict),
    }
    model = project.log_model(name="cancer_classifier",
                              kind="sklearn",
                              source="./model/")
    model.log_metrics(metrics)
    return model

In [ ]:
train_fn = project.new_function(name="train",
                                kind="python",
                                python_version="PYTHON3_10",
                                code_src="src/train-model.py",
                                handler="train",
                                requirements=["numpy<2"])

In [ ]:
train_run = train_fn.run(action="job",
                         inputs={"di": gen_data_run.output("dataset").key},
                         wait=True)

Wait until the train job run is completed. The state of the run can be viewed using console or via digitalhub sdk API call

In [ ]:
train_run.refresh().status.state

Once training is 'Completed', proceed with  serve.

## Serve

In [ ]:
model = train_run.output("model")

In [ ]:
func = project.new_function(name="serve_sklearnmodel",
                            kind="sklearnserve",
                            path=model.spec.path + 'cancer_classifier.pkl')

serve_run = func.run("serve",
                     labels=["service_sklearn"],
                     wait=True)

Once 'Service' object available, proceed by making call to deployed service.

In [ ]:
import numpy as np

data = np.random.rand(2, 30).tolist()
json = {
    "inputs": [
        {
        "name": "input-0",
        "shape": [2, 30],
        "datatype": "FP32",
        "data": data
        }
    ]
}

In [ ]:
serve_run.refresh().invoke(json=json).json()

## Pipeline

In [ ]:
%%writefile "src/pipeline.py"
from hera.workflows import Workflow, DAG, Parameter
from digitalhub_runtime_hera.dsl import step


def pipeline():
    with Workflow(entrypoint="dag") as w:
        with DAG(name="dag"):
            A = step(template={"action":"job"},
                     function="download-prep",
                     outputs=["dataset"])
            B = step(template={"action":"job", "inputs": {"di": "{{inputs.parameters.di}}"}},
                     function="train",
                     inputs={"di": A.get_parameter("dataset")})
            A >> B
    return w


In [ ]:
workflow = project.new_workflow(name="pipeline_ml",
                                kind="hera",
                                code_src="src/pipeline.py",
                                handler="pipeline")

In [ ]:
build_run = workflow.run("build", wait=True)

In [ ]:
wf_run = workflow.run("pipeline", wait=True)

The pipeline does not include the 'serve' phase of scenario.